# NYC: Hospitals to Neighbourhoods

### In this notebook, the data collected about the hospitals, will be used to determine how much hospital care is available for each neighbourhood

The parameters calculared will be:

1- The average distance between the neghbourhood and every available hospital in NYC

2- The number of hospitals available within a radius of 2KM from the neghbourhood

### Importing necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import geopy.distance
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

### Loading hospitals data and selecting the unique occurances

In [2]:
NYHospitals=pd.read_csv('NYHospitals.csv')
UnHos=NYHospitals.drop_duplicates()
UnHos.shape

(1039, 9)

In [3]:
UnHos.head()

,name,categories,lat,lng,id,city,state,formattedAddress,labeledLatLngs
0,Montefiore Medical Center - Wakefield Campus,Hospital,40.894207,-73.860790,4addedeaf964a520646621e3,Bronx,NY,"['600 E 233rd St (Carpenter Ave.)', 'Bronx, NY...","[{'label': 'display', 'lat': 40.89420660561763..."
1,Montefiore Medical Pavillion,Medical Center,40.880135,-73.878712,4db854784b22f2ddb6399a1b,Bronx,NY,"['3400 Bainbridge Ave', 'Bronx, NY 10467', 'Un...","[{'label': 'display', 'lat': 40.88013478363959..."
2,Montefiore Medical Center Mount Vernon,Hospital,40.913123,-73.840528,4c051aa973a8c9b6a7f196e0,Mount Vernon,NY,"['12 N 7th Ave', 'Mount Vernon, NY 10550', 'Un...","[{'label': 'display', 'lat': 40.91312340197153..."
3,The Children’s Hospital at Montefiore,Hospital,40.880640,-73.879153,4c3e307edb3b1b8d56106595,Bronx,NY,"['3415 Bainbridge Ave (at E Gun Hill Rd)', 'Br...","[{'label': 'display', 'lat': 40.88063954722417..."
4,North Central Bronx Hospital,Hospital,40.880441,-73.881526,4b6de6a8f964a520439a2ce3,Bronx,NY,"['3224 Kossuth Ave (at E 208th St)', 'Bronx, N...","[{'label': 'display', 'lat': 40.88044133267162..."


## Loading the neghbourhood data

In [4]:
NYNeighborhoods=pd.read_csv('NYNeighborhoods.csv')
NYNeighborhoods.shape

(306, 4)

In [5]:
NYNeighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Performing the data analysis

In the following cell:

1- The average distance between the neighbourhood and each hospital is calculated

2- The number of hospitals within a 2Km range is counted

In [6]:
AvgDist=pd.DataFrame([])
NearHos=pd.DataFrame([])
for Nbrhd,LatN,LngN in zip(NYNeighborhoods['Neighborhood'],NYNeighborhoods['Latitude'],NYNeighborhoods['Longitude']):
    Counter=0
    TotalDist=0
    for LatH,LngH in zip(UnHos['lat'],UnHos['lng']):
        NN = (LatN, LngN)
        HH = (LatH, LngH)
        Distance= geopy.distance.vincenty(NN, HH).km
        TotalDist=TotalDist+Distance
        if Distance<=2:
            Counter+=1
    AvgDist=AvgDist.append([TotalDist/UnHos.shape[0]])
    NearHos=NearHos.append([Counter])

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  if __name__ == '__main__':


## The calculated quantities are combined in a dataframe to be joined with the neghbourhoods'dataframe

In [7]:
NewDf=pd.concat([NearHos, AvgDist], axis=1)
NewDf.columns=['NearHos','AvgDist']
NewDf=NewDf.reset_index(drop=True)
NewDf.tail()

,NearHos,AvgDist
301,27,12.264738
302,0,22.586070
303,3,22.610268
304,93,11.069510
305,3,20.389055


In [8]:
NYNeighborhoods=NYNeighborhoods.join(NewDf, how='inner')
NYNeighborhoods.tail()

,Borough,Neighborhood,Latitude,Longitude,NearHos,AvgDist
301,Manhattan,Hudson Yards,40.756658,-74.000111,27,12.264738
302,Queens,Hammels,40.587338,-73.805530,0,22.586070
303,Queens,Bayswater,40.611322,-73.765968,3,22.610268
304,Queens,Queensbridge,40.756091,-73.945631,93,11.069510
305,Staten Island,Fox Hills,40.617311,-74.081740,3,20.389055


## The results are saved in a CSV file for further use

In [116]:
NYNeighborhoods.to_csv('NYNeighborhoodsWithHospitals.csv', index=False)

## The data analysis is perfrmed on the next notebook:
## NY Neighborhood Populations